In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

random.seed(777)
torch.manual_seed(777)

In [2]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [3]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = 'data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [4]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [5]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias = True)
linear2 = torch.nn.Linear(256, 256, bias = True)
linear3 = torch.nn.Linear(256, 10, bias = True)
relu = torch.nn.ReLU()

In [6]:
# initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [7]:
# model
model = torch.nn.Sequential(linear1, relu,
                            linear2, relu,
                            linear3)

In [8]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# training
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        # label is not one-hot encoded
        X = X.view(-1, 28*28)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 129.392074585
Epoch: 0002 cost = 35.814987183
Epoch: 0003 cost = 22.769479752
Epoch: 0004 cost = 15.833213806
Epoch: 0005 cost = 11.565637589
Epoch: 0006 cost = 8.528262138
Epoch: 0007 cost = 6.380707264
Epoch: 0008 cost = 4.734078407
Epoch: 0009 cost = 3.576365471
Epoch: 0010 cost = 2.668595314
Epoch: 0011 cost = 2.104521751
Epoch: 0012 cost = 1.622449875
Epoch: 0013 cost = 1.187595606
Epoch: 0014 cost = 1.028596878
Epoch: 0015 cost = 0.773602426


In [12]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28*28).float()
    Y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

Accuracy :  0.9435999989509583
